In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
#we create image generator function to genarate training and test as well as to automatically label the data

def image_gen_w_aug(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 30,  
                                      zoom_range = 0.2, 
                                      width_shift_range=0.1,  
                                      height_shift_range=0.1,
                                      validation_split = 0.15)
    
  
    
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                       target_size = (75,75),
                                                       batch_size = 214,
                                                       class_mode = 'categorical',
                                                       subset='training')
    
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                          target_size = (75,75),
                                                          batch_size = 37,
                                                          class_mode = 'categorical',
                                                          subset = 'validation')
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                     target_size=(75,75),
                                                     batch_size = 37,
                                                     class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator

In [3]:
def model_output_for_TL (pre_trained_model, last_output):

    x = Flatten()(last_output)
    
    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output neuron. 
    x = Dense(3, activation='softmax')(x)
    
    model = Model(pre_trained_model.input, x)
    
    return model

In [6]:
train_dir = os.path.join('D:/Projects/AI/rps-class/dataset/rps/rps/')
test_dir = os.path.join('D:/Projects/AI/rps-class/dataset/rps/rps')

In [7]:
train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)

Found 2142 images belonging to 3 classes.
Found 378 images belonging to 3 classes.
Found 2520 images belonging to 3 classes.


In [8]:
pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

In [9]:
model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history_TL = model_TL.fit(
      train_generator,
      steps_per_epoch=10,  
      epochs=20,
      verbose=1,
      validation_data = validation_generator)

tf.keras.models.save_model(model_TL,'my_model.hdf5')

Epoch 1/20
10/10 [==============================] - 28s 3s/step - loss: 15.3492 - accuracy: 0.3417 - val_loss: 0.8027 - val_accuracy: 0.5952
Epoch 2/20
10/10 [==============================] - 11s 1s/step - loss: 1.5521 - accuracy: 0.6178 - val_loss: 1.0518 - val_accuracy: 0.6058
Epoch 3/20
10/10 [==============================] - 11s 1s/step - loss: 0.6912 - accuracy: 0.7705 - val_loss: 0.2047 - val_accuracy: 0.9206
Epoch 4/20
10/10 [==============================] - 11s 1s/step - loss: 0.2413 - accuracy: 0.9018 - val_loss: 0.2249 - val_accuracy: 0.9048
Epoch 5/20
10/10 [==============================] - 11s 1s/step - loss: 0.1173 - accuracy: 0.9684 - val_loss: 0.1384 - val_accuracy: 0.9550
Epoch 6/20
10/10 [==============================] - 11s 1s/step - loss: 0.0965 - accuracy: 0.9763 - val_loss: 0.1590 - val_accuracy: 0.9418
Epoch 7/20
10/10 [==============================] - 11s 1s/step - loss: 0.0647 - accuracy: 0.9893 - val_loss: 0.1508 - val_accuracy: 0.9550
Epoch 8/20
10/10 [=